In [26]:
import os
import json
import pandas as pd
import traceback

In [27]:
from langchain import PromptTemplate, LLMChain
from langchain_cohere import ChatCohere
from langchain_core.messages import HumanMessage

In [28]:
from dotenv import load_dotenv
load_dotenv()

True

In [83]:
KEY = os.getenv('COHERE_API_TOKEN')

In [84]:
llm = ChatCohere(cohere_api_key=KEY)

In [85]:
llm

ChatCohere(client=<cohere.client.Client object at 0x000001FA8F8EDE50>, async_client=<cohere.client.AsyncClient object at 0x000001FA8F8EDC10>, cohere_api_key=SecretStr('**********'))

In [86]:
from langchain.chains import SequentialChain
import PyPDF2

In [87]:
RESPONSE_JSON = {
    "1": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "2": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
    "3": {
        "mcq": "multiple choice question",
        "options": {
            "a": "choice here",
            "b": "choice here",
            "c": "choice here",
            "d": "choice here",
        },
        "correct": "correct answer",
    },
}

In [88]:
json.dumps(RESPONSE_JSON)

'{"1": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "2": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}, "3": {"mcq": "multiple choice question", "options": {"a": "choice here", "b": "choice here", "c": "choice here", "d": "choice here"}, "correct": "correct answer"}}'

In [145]:
TEMPLATE="""
Text:{text}
You are an expert MCQ maker. Given the above text, it is your job to \
create a quiz  of {number} multiple choice questions for {subject} students in {tone} tone. 
Make sure the questions are not repeated and check all the questions to be conforming the text as well.
Make sure to format your response like  RESPONSE_JSON below and use it as a guide and make sure that the formate is in json  that are able to loads in json. \
Do not wrtie json and \\n in response. Ensure to make {number} MCQs
### RESPONSE_JSON
{response_json}

"""

In [146]:
quiz_generation_prompt = PromptTemplate(
    input_variables=["text", "number", "subject", "tone", "response_json"],
    template=TEMPLATE
    )

In [147]:
quiz_chain=LLMChain(llm=llm, prompt=quiz_generation_prompt, output_key="quiz", verbose=True)

In [148]:
TEMPLATE2="""
You are an expert english grammarian and writer. Given a Multiple Choice Quiz for {subject} students.\
You need to evaluate the complexity of the question and give a complete analysis of the quiz. Only use at max 50 words for complexity analysis. 
if the quiz is not at per with the cognitive and analytical abilities of the students,\
update the quiz questions which needs to be changed and change the tone such that it perfectly fits the student abilities
Quiz_MCQs:
{quiz}

Check from an expert English Writer of the above quiz:
"""

In [149]:
quiz_evaluation_prompt=PromptTemplate(input_variables=["subject", "quiz"], template=TEMPLATE)

In [150]:
review_chain=LLMChain(llm=llm, prompt=quiz_evaluation_prompt, output_key="review", verbose=True)

In [151]:
generate_evaluate_chain=SequentialChain(chains=[quiz_chain, review_chain], input_variables=["text", "number", "subject", "tone", "response_json"],
                                        output_variables=["quiz", "review"], verbose=True,)

In [152]:
file_path = r"C:\Users\shail\Desktop\Projects\mcq_generator\data.txt"

In [153]:
with open(file_path, 'r') as file:
    TEXT = file.read()

In [154]:
NUMBER=5 
SUBJECT="biology"
TONE="simple"

In [155]:
response=generate_evaluate_chain(
        {
            "text": TEXT,
            "number": NUMBER,
            "subject":SUBJECT,
            "tone": TONE,
            "response_json": json.dumps(RESPONSE_JSON)
        }
)



> Entering new SequentialChain chain...


> Entering new LLMChain chain...
Prompt after formatting:

Text:Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]

Biologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each define

In [156]:
response

{'text': 'Biology is the scientific study of life.[1][2][3] It is a natural science with a broad scope but has several unifying themes that tie it together as a single, coherent field.[1][2][3] For instance, all organisms are made up of cells that process hereditary information encoded in genes, which can be transmitted to future generations. Another major theme is evolution, which explains the unity and diversity of life.[1][2][3] Energy processing is also important to life as it allows organisms to move, grow, and reproduce.[1][2][3] Finally, all organisms are able to regulate their own internal environments.[1][2][3][4][5]\n\nBiologists are able to study life at multiple levels of organization,[1] from the molecular biology of a cell to the anatomy and physiology of plants and animals, and evolution of populations.[1][6] Hence, there are multiple subdisciplines within biology, each defined by the nature of their research questions and the tools that they use.[7][8][9] Like other sci

In [157]:
quiz=response.get("quiz")

In [159]:
quiz = json.loads(quiz)

In [160]:
quiz_table_data = []
for key, value in quiz.items():
    mcq = value["mcq"]
    options = " | ".join(
        [
            f"{option}: {option_value}"
            for option, option_value in value["options"].items()
            ]
        )
    correct = value["correct"]
    quiz_table_data.append({"MCQ": mcq, "Choices": options, "Correct": correct})

In [161]:
quiz_table_data

[{'MCQ': 'What is the main focus of the scientific study of Biology?',
  'Choices': 'a: The behavior of living organisms | b: The structure and composition of cells | c: The evolution and diversity of life | d: The chemical processes that sustain life',
  'Correct': 'c'},
 {'MCQ': 'Which of the following is NOT a unifying theme in Biology?',
  'Choices': 'a: Cellular organization and function | b: Genetics and heredity | c: Biochemical processes in organisms | d: Environmental regulation in organisms',
  'Correct': 'a'},
 {'MCQ': 'What is the role of energy processing in living organisms?',
  'Choices': 'a: It allows for movement and growth | b: It is necessary for reproduction and development | c: It helps in the regulation of internal body temperature | d: All of the above',
  'Correct': 'd'},
 {'MCQ': 'Which level of organization does Molecular Biology focus on?',
  'Choices': 'a: Cells | b: Tissues | c: Organs | d: Organ Systems',
  'Correct': 'a'},
 {'MCQ': 'What are the two main 

In [162]:
quiz=pd.DataFrame(quiz_table_data)


In [163]:
quiz.to_csv("mcq_questions.csv",index=False)
